###  Using API to run the simulation

We use a strongly defined HTTP API protocol to communicate with a C12 backend (get and receive the data). This tutorial will briefly present basic API calls that can be used for communication with the C12 backend. Our APIs are the low-level access to our simulator, and could be made compatible with other quantum computer frameworks and interfaces.

To run each API, we need to pass the unique bearer token that will be used to verify the user's right to access C12's system.

For communication with C12's system, it is possible to use the API library that encapsulates the HTTP communication with the C12 system using the python request module.

To start using it, we need to create an instance of the Request class, as shown in the following code snippet.

In [2]:
# Installing the c12simulator-clients package (for more details see jupyter 1)
import os
from c12simulator_clients.api.client import Request

TOKEN = os.getenv("C12_TOKEN")


# Create the request instance
# Constructor of the Request class also accepts the verbose parameter, which can be use for more detailed output of the methods.

request = Request(auth_token=TOKEN, verbose=False)

#### Run the simulation

The method `start_job()` is used to start the simulation. It has the following arguments:
- `qasm_str`: QASM string with quantum circuit
- `shots`:  Number of trials for the simulation
- `result`: what is desired output (statevector, counts, density_matrix)
- `backend_name`: the name of the backend to run on

In [3]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)


qasm_str = circuit.qasm()
res = request.start_job(
    qasm_str=qasm_str, shots=100, result="density_matrix", backend_name="c12sim-iswap"
)

# The function returns the UUID and QasmStr of started job or HTTP error in case of problems
print(res)
job_uuid = res[0]


('9866fd4a-793d-4641-a264-9f4c1bff96d6', 'OPENQASM 2.0;\ninclude "qelib1.inc";\ngate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }\nqreg q[2];\nry(-pi/2) q[1];\niswap q[0],q[1];\nrx(pi) q[0];\nry(-pi/2) q[1];\niswap q[0],q[1];\nry(-pi/2) q[0];\nrz(-pi/2) q[0];\nrz(pi/2) q[1];\nry(-pi/2) q[1];\n')


#### Get the results of a simulation

To get the simulation results, we can use the `get_job_result()` method that accepts one mandatory argument (job UUID string) and two optional ones.
- `timeout`, which represents the number of seconds that the function will wait for job completion
- `wait`, number of seconds between each check if the job is finished

If we do not specify a timeout argument the function will run until the result is obtained

In [11]:
job_result = request.get_job_result(job_uuid, timeout=120, wait=5)
print(job_result)

{'status': 'FINISHED', 'results': {'density_matrix': [['(0.5000897437714247+0j)', '(-5.274116340186169e-05+5.116335835910993e-05j)', '(-5.6216557959268616e-05+1.063111113359385e-05j)', '(0.49999998325729417+4.014865527146583e-06j)'], ['(-5.274116340186169e-05-5.116335835910993e-05j)', '(1.0796701237753112e-08+0j)', '(7.016440672111406e-09+4.63023039361485e-09j)', '(-5.273128619880364e-05-5.115459853004587e-05j)'], ['(-5.6216557959268616e-05-1.063111113359385e-05j)', '(7.016440672111406e-09-4.63023039361485e-09j)', '(6.5454689952985004e-09+0j)', '(-5.620638236640756e-05-1.0629654290917945e-05j)'], ['(0.49999998325729417-4.014865527146583e-06j)', '(-5.273128619880364e-05+5.115459853004587e-05j)', '(-5.620638236640756e-05+1.0629654290917945e-05j)', '(0.4999102388864063+0j)']]}, 'errors': ''}


#### Get the status of the job

In [ ]:
status = request.get_job_status(job_uuid=job_uuid)
print(status)

#### Get all jobs

Method `get_user_jobs()` can be used for obtaining information about all the jobs that have been run on the system for the user whose token is passed to the Request instance.

The method has two arguments:
- `limit`: an integer that represents the number of results to obtain
- `offset`: an integer that represents the offset from the first result

Using these two values, one can obtain the pagination, where the offset represents a page and limits the number of results per page. Method returns an array with results.


In [14]:
number_of_results = 5  # Limit parameter, number of records to retrieve
offset = 0
count = 1
while True:
    jobs = request.get_user_jobs(limit=number_of_results, offset=offset)

    for job in jobs:
        print(f"{count:}")
        print(job)
        count += 1

    if len(jobs) < number_of_results:
        break

    offset += number_of_results

1
{'uuid': '133c7598-6fe6-4067-bb12-1ab444a6dee2', 'time': '2023-03-23T16:54:26.284840', 'status': 'finished', 'options': {'shots': 1024, 'result': 'counts,statevector'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nry(pi/2) q[0];\nrx(-1.8157749899217606) q[1];\nrz(-pi) q[1];\ncrx(pi) q[0],q[1];\nrz(pi/2) q[0];\nry(pi) q[0];\nrz(-pi) q[1];\nrx(-1.3258176636680312) q[1];\n', 'task_orig': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nh q[0];\ncx q[0],q[1];\n'}
2
{'uuid': '7ba5a2f4-9d34-4964-91d1-ceeee047be53', 'time': '2023-03-23T16:59:49.771009', 'status': 'finished', 'options': {'shots': 10000, 'result': 'counts,statevector'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nry(-pi/2) q[0];\nrz(-pi) q[0];\nrx(1.815774989921761) q[1];\ncrx(pi) q[0],q[1];\nrz(pi/2) q[0];\nry(-pi/2) q[0];\nrz(-2.3713483251754366) q[1];\nry(0.3447011797986471) q[1];\nrz(-2.3713483251754415) q[1];\ncrx(pi) q[0],q[1];\nrz(-pi/2) q[0];\nry(-pi/2) q[0];\nrz(-pi) q[1];\nrx(-1.325817

#### Get information about a specific job

Using the method `get_job()` it is possible to retrieve the information about the specific job with a given UUID.

In [17]:
job_data = request.get_job(job_uuid)
print(job_data)

{'uuid': '7cb69177-cd38-4a7f-8816-5d9c8e5d8388', 'time': '2023-03-24T10:51:18.155981', 'status': 'running', 'options': {'shots': 100, 'result': 'density_matrix'}, 'task': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nry(pi/2) q[0];\nrx(-1.8157749899217606) q[1];\nrz(-pi) q[1];\ncrx(pi) q[0],q[1];\nrz(pi/2) q[0];\nry(pi) q[0];\nrz(-pi) q[1];\nrx(-1.3258176636680312) q[1];\n', 'task_orig': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[2];\nh q[0];\ncx q[0],q[1];\n'}
